In [14]:
UMLS_ST21pv_ontologies_ids = ['CPT', 'FMA', 'GO', 'HGNC', 'HPO', 'ICD10', 'ICD10CM', 'ICD9CM', 'MDR', 'MSH', 'MTH', 'NCBI', 'NCI', 'NDDF', 'NDFRT', 'OMIM', 'RXNORM', 'SNOMEDCT_US']

# HPO = HP
# MDR = MEDDRA
# MSH = MESH
# MTH = ?
# NCBI = NCBITAXON
# NCI = NCIT
# SNOMEDCT_US = SNMI
UMLS_NCBO_ST21pv_ontologies_ids = ['CPT', 'FMA', 'GO', 'HGNC', 'HP', 'ICD10', 'ICD10CM', 'ICD9CM', 'MEDDRA', 'MESH', 'NCBITAXON', 'NCIT', 'NDDF', 'NDFRT', 'OMIM', 'RXNORM', 'SNMI']


UMLS_ST21pv_semantic_types_ids = ['T005', 'T007', 'T017', 'T022', 'T031', 'T033', 'T037', 'T038',
'T058', 'T062', 'T074', 'T082', 'T091', 'T092', 'T097', 'T098', 'T103', 'T168', 'T170', 'T201', 'T204']

unique_concept_ids_path = "../data/unique_concepts_id.txt"
concept_names_results_path = "../data/concept_names.csv"

In [2]:
import urllib.request, urllib.error, urllib.parse
import json
REST_URL = "http://data.bioontology.org"
API_KEY = "194c9635-ce67-4e70-81c5-898c3a2b30fb"

def get_json(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    return json.loads(opener.open(url).read())

In [3]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
def process_chunk(concepts_id_names, page_link):
    while page_link:
        result = get_json(page_link)
        # extract pairs (cui, concept_name)
        concepts = result['collection']
        for concept in concepts:
            concept_cuis = concept['cui']
            concept_name = concept['prefLabel']
            for cui in concept_cuis:
                pass
                concepts_id_names.append([cui, concept_name])
        
        page_link = result['links']['nextPage']

In [5]:
import numpy as np

with open(unique_concept_ids_path) as f:
    line = f.read()
concept_ids = np.array(line.split(','))

concept_ids[-1] = concept_ids[-1].replace('\n', '')
chunked_concpet_ids = list(chunks(concept_ids, 500))

In [ ]:
concepts_id_names = []
ontologies_query = f"ontologies={','.join(UMLS_NCBO_ST21pv_ontologies_ids)}"
for chunk in chunked_concpet_ids:
    cui_query = f"cui={','.join(chunk)}"
    query = f"https://data.bioontology.org/search?{ontologies_query}&{cui_query}&apikey={API_KEY}"
    process_chunk(concepts_id_names, query)
    print(len(concepts_id_names))

In [37]:
import pandas as pd
res_df = pd.DataFrame(concepts_id_names, columns=['concept_id','concept_name'])
res_df.concept_name = res_df.concept_name.replace('"', '')

In [38]:
res_df.to_csv(concept_names_results_path, index=False)